In [1]:
!pip install tensorflow==2.15.1 hexdump Jinja2 eloquent_tensorflow odfpy -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.0 requires protobuf<4,>=3.11, but you have protobuf 4.25.5 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from eloquent_tensorflow import convert_model
import numpy as np

# Path to your dataset file
file_path = "/content/dataset.csv"

# Read the dataset
df = pd.read_csv(file_path)

# Define column names based on the data collected in your sampler code
df.columns = [
    "control_value",    # Target variable
    "proximity",
    "ambient",
    "red_LSB",
    "green_LSB",
    "blue_LSB",
    "white_LSB",
    "cct_LSB",
    "lux_LSB",
    "red_LSA",
    "green_LSA",
    "blue_LSA",
    "white_LSA",
    "cct_LSA",
    "lux_LSA"
]

# Display the first few rows to verify data loading
print(df.head())

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Separate features (X) and target (y)
X = df.drop(columns=['control_value']).values  # All columns except 'control_value'
y = df['control_value'].values  # The target variable

# Optionally, you can print the shapes to verify
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

You'll need to hardcode these min and max values into your inference code on the ESP32.

In [ ]:
# Fit the scaler on the entire dataset (before splitting)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Extract the min and max values
data_min = scaler.data_min_
data_max = scaler.data_max_

print("Data Min:", data_min)
print("Data Max:", data_max)

In [ ]:
# Split the data into training and testing sets
# Use shuffle=False if your data is time-series or sequential
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, shuffle=True
)

In [ ]:
# Define the model architecture
input_shape = (X_train.shape[1],)  # Number of features

model = tf.keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

# Train the model with callbacks
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=1000,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model on the test set
print("\nEvaluate on the test set:")
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")
print(f"Test Loss: {test_loss}")

# Convert and print the model for deployment
print("\nConverting the model for deployment:")
print(convert_model(model))

Epoch 1/200
783/783 [==============================] - 7s 8ms/step - loss: 695.6523 - mean_absolute_error: 7.1752 - val_loss: 6.4433 - val_mean_absolute_error: 0.9895
Epoch 2/200
783/783 [==============================] - 5s 7ms/step - loss: 5.0292 - mean_absolute_error: 0.8809 - val_loss: 3.9358 - val_mean_absolute_error: 0.7676
Epoch 3/200
783/783 [==============================] - 4s 5ms/step - loss: 2.6165 - mean_absolute_error: 0.6420 - val_loss: 1.8438 - val_mean_absolute_error: 0.5528
Epoch 4/200
783/783 [==============================] - 2s 3ms/step - loss: 1.0700 - mean_absolute_error: 0.4771 - val_loss: 0.7547 - val_mean_absolute_error: 0.4312
Epoch 5/200
783/783 [==============================] - 3s 4ms/step - loss: 0.5503 - mean_absolute_error: 0.4119 - val_loss: 0.4887 - val_mean_absolute_error: 0.3976
Epoch 6/200
783/783 [==============================] - 2s 3ms/step - loss: 0.4187 - mean_absolute_error: 0.3822 - val_loss: 0.4115 - val_mean_absolute_error: 0.3940
Epoch 7/

In [21]:
print(convert_model(model))


#pragma once

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

// automatically configure network
#define TF_NUM_INPUTS 4
#define TF_NUM_OUTPUTS 1
#define TF_NUM_OPS 2

/**
 * Call this function to register the ops
 * that have been detected
 */
template<class TF>
void registerNetworkOps(TF& nn) {
    nn.resolver.AddFullyConnected();
    nn.resolver.AddSoftmax();
    
}




// model data
const unsigned char tfModel[14108] DATA_ALIGN_ATTRIBUTE = { 0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x90, 0x00, 0x00, 0x00, 0xe8, 0x00, 0x00, 0x00, 0x28, 0x30, 0x00, 0x00, 0x38, 0x30, 0x00, 0x00, 0x